In [ ]:
import random
def Chromosome_Structure():
      return {"stop_loss": round(random.randint(1, 99)),
            "take_profit": round(random.randint(1,99)),
            "trade_size": round(random.randint(1, 99)),
            }

def initialize_population(p_size):
      population = []
      for i in range(p_size):
           population.append(Chromosome_Structure())
      return population

def evaluate_fitness(strategy,price_changes):
    capital = 1000
    stop_loss = strategy["stop_loss"] / 100
    take_profit = strategy["take_profit"] / 100
    trade_size = strategy["trade_size"] / 100
    curr_capital = capital

    for c in price_changes:
        trade_amount = curr_capital * trade_size
        c_percentage = c / 100

        if c >= take_profit * 100:
            profit = trade_amount * take_profit
        elif c <= -stop_loss * 100:
            profit = -trade_amount * stop_loss
        else:
            profit = trade_amount * c_percentage

        curr_capital += profit

    return round(curr_capital - capital, 2)

def parents_select(population):
    return random.sample(population, 2)

def single_point_crossover(p1, p2):
    child1,child2 = p1.copy(), p2.copy()
    crossover_point = random.choice(["stop_loss", "take_profit", "trade_size"])
    child1[crossover_point], child2[crossover_point] = child2[crossover_point], child1[crossover_point]
    return child1, child2
#task-03________________________________________________________________________
def two_point_crossover(p1, p2):
    keys = list(p1.keys())
    point1 = random.randint(1, len(keys) - 2)
    point2 = random.randint(point1 + 1, len(keys) - 1)
    child1 = {}
    child2 = {}
    for i, key in enumerate(keys):
        if i < point1 or i >= point2:
            child1[key] = p1[key]
            child2[key] = p2[key]
        else:
            child1[key] = p2[key]
            child2[key] = p1[key]

    return child1, child2
#_______________________________________________________________________________


def mutate(chromosome):
    mutation_rate = 0.05
    if random.random() < mutation_rate:
        value = random.choice(["stop_loss", "take_profit", "trade_size"])
        chromosome[value] = random.randint(1, 99)
    return chromosome

def GA(generation, p_size, hist_price):
    population = initialize_population(p_size)

    for k in range(generation):
        for i in range(len(population)):
            for j in range(i + 1, len(population)):
                fitness_i = evaluate_fitness(population[i], hist_price)
                fitness_j = evaluate_fitness(population[j], hist_price)
                if fitness_i < fitness_j:  # Swap if i is worse than j
                    population[i], population[j] = population[j], population[i]
        new_population = population[:2]

        while len(new_population) < p_size:
            p1, p2 = parents_select(population)
            c1, c2 = single_point_crossover(p1, p2)
      #      c1 = two_point_crossover(p1, p2)[0]  for task 03
      #      c2 = two_point_crossover(p1, p2)[1]  for task03
      #      print(c1,c2)
            mutate(c1)
            mutate(c2)
            new_population.append(c1)
            new_population.append(c2)

        population = new_population[:p_size]

    # Find the best strategy at the end
    best_strategy = population[0]  # Since it's sorted, first one is best
    best_fitness = evaluate_fitness(best_strategy, hist_price)
    return best_strategy, best_fitness
historical_prices = [-1.2, 3.4, -0.8, 2.1, -2.5, 1.7, -0.3, 5.8, -1.1, 3.5]
initial_population = [
    {"stop_loss": 2, "take_profit": 5, "trade_size": 20},
    {"stop_loss": 3, "take_profit": 7, "trade_size": 30},
    {"stop_loss": 1.5, "take_profit": 4, "trade_size": 25},
    {"stop_loss": 2.5, "take_profit": 6, "trade_size": 15},
]

generation = 10
best_strategy, final_profit = GA(generation, len(initial_population), historical_prices)
print(f"Best stragey:{ best_strategy}")
print(f"Final Profit:{final_profit}")

Best stragey:{'stop_loss': 10, 'take_profit': 85, 'trade_size': 96}
Final Profit:103.41
